In [905]:
''' 
   Size = 28x28
   Representation : 28x28 = 
   Training set : 60,000
   Test set : 10,000       '''

' \n   Size = 28x28\n   Representation : 28x28 = \n   Training set : 60,000\n   Test set : 10,000       '

In [906]:
#imports

import tensorflow as tf
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Import MNIST data
#http://yann.lecun.com/exdb/mnist/
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/", one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [907]:
#Write function to reset session

def tf_reset():
    try:
        sess.close()
    except:
        pass
    tf.reset_default_graph()
    return tf.Session()


In [908]:
#Call the reset function

sess = tf_reset()

In [909]:
#Define input_placeholder

input_ph = tf.placeholder(dtype=tf.float32, shape=[None, 784]) #inputs 28X28=784

In [910]:
#Define output_placeholder

n_classes = 10
output_ph = tf.placeholder(dtype=tf.float32, shape=[None, n_classes]) #MNIST classes


In [911]:
#Store weigth and biases


def get_weights_bias(n_classes): 
    w = {
        "w1" : tf.get_variable('W0', shape=(5,5,1,32), initializer=tf.contrib.layers.xavier_initializer()), 
        "w2" : tf.get_variable('W1', shape=(5,5,32,64), initializer=tf.contrib.layers.xavier_initializer()),
        'wd1': tf.get_variable('W3', shape=(7*7*64,1024), initializer=tf.contrib.layers.xavier_initializer()), 
        'out': tf.get_variable('W4', shape=(1024,10), initializer=tf.contrib.layers.xavier_initializer())
    }

    


    b = {
        "b1" : tf.get_variable('B0', shape=(32), initializer=tf.contrib.layers.xavier_initializer()),
        "b2" : tf.get_variable('B1', shape=(64), initializer=tf.contrib.layers.xavier_initializer()),
        "b4" : tf.get_variable('B3', shape=(1024), initializer=tf.contrib.layers.xavier_initializer()),
        "b5" : tf.get_variable('B4', shape=(10), initializer=tf.contrib.layers.xavier_initializer())
    }

    return w,b

In [912]:
#Convolve, MaxPool, FullyConnect

def conv2d(X,w,b,strides=1):
    #building conv-layer
    x = tf.nn.conv2d(X,w,strides=[1,strides,strides,1],padding="SAME")
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

def maxpool2d(X, k=2): #k defines how much you want to reduce your matrix
    return tf.nn.max_pool(X, ksize=[1, k, k, 1], strides=[1, k, k, 1],padding='SAME')

def fully_connected(X,w,b): #reduce from 2d to 1d
    fc = tf.reshape(X, [-1,w.get_shape().as_list()[0]])
    fc = tf.matmul(fc, w)
    fc = tf.add(fc,b)
    return tf.nn.relu(fc)



In [913]:
#Build a model


def build_model(input_layer,w,b):

    input_layer = tf.reshape(input_layer,[-1,28,28,1])

    #conv-layer-1
    conv1 = conv2d(input_layer,w['w1'],b['b1'])
    conv1 = maxpool2d(conv1, k=2)
    print(conv1.get_shape())

    conv2 = conv2d(conv1,w['w2'],b['b2'])
    conv2 = maxpool2d(conv2, k=2)
    print(conv2.get_shape())

    fc1 = fully_connected(conv2,w['wd1'],b['b4'])
    print(fc1.get_shape())

    out = tf.matmul(fc1,w['out'])

    return tf.add(out,b['b5'])


In [914]:
#Output_pred = build_model(input_layer,w,b)      

w,b = get_weights_bias(10)                                                 
output_pred = build_model(input_ph,w,b)          
sess.run(tf.global_variables_initializer())

(?, 14, 14, 32)
(?, 7, 7, 64)
(?, 1024)


In [915]:
#Define Loss function

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output_pred,labels=output_ph))

In [916]:
#Define Optimizer

opt = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

In [917]:
#Run session to compute loss

pred = tf.nn.softmax(output_pred)
sess.run(tf.global_variables_initializer())
print (pred)

Tensor("Softmax:0", shape=(?, 10), dtype=float32)


In [918]:
#Try for accuracy

actual = tf.equal(tf.argmax(pred,1),tf.argmax(output_ph,1))
accuracy = tf.reduce_mean(tf.cast(actual,tf.float32))


In [919]:
#Initialise you variables

batch_size = 128

In [920]:
#Iterate and check if loss reduces

for i in range(100):
    batch_input , batch_output = mnist.train.next_batch(batch_size)
    _,mse = sess.run([opt,loss],feed_dict={input_ph: batch_input, output_ph: batch_output})
    
    
#print loss w.r.t current iteration value    
    
    mse,acr = sess.run([loss,accuracy],feed_dict={input_ph:batch_input, output_ph:batch_output})
    print("iteration : {}, loss : {}, accuracy : {}".format(i,mse,acr))
    

iteration : 0, loss : 18829.76171875, accuracy : 0.1484375
iteration : 1, loss : 329.94708251953125, accuracy : 0.1015625
iteration : 2, loss : 14.337419509887695, accuracy : 0.109375
iteration : 3, loss : 3.6372578144073486, accuracy : 0.1171875
iteration : 4, loss : 4.120210647583008, accuracy : 0.1328125
iteration : 5, loss : 2.7559189796447754, accuracy : 0.15625
iteration : 6, loss : 2.341729164123535, accuracy : 0.140625
iteration : 7, loss : 2.594339370727539, accuracy : 0.125
iteration : 8, loss : 2.5110533237457275, accuracy : 0.1015625
iteration : 9, loss : 2.3434762954711914, accuracy : 0.125
iteration : 10, loss : 2.3762190341949463, accuracy : 0.109375
iteration : 11, loss : 2.3954625129699707, accuracy : 0.1484375
iteration : 12, loss : 2.4553308486938477, accuracy : 0.0625
iteration : 13, loss : 2.3662376403808594, accuracy : 0.09375
iteration : 14, loss : 2.336679697036743, accuracy : 0.1171875
iteration : 15, loss : 2.300668239593506, accuracy : 0.09375
iteration : 16,